# Export csv file(behavior_ms) from txt file

In [ ]:
import csv
import sys
import os
import re
import pandas as pd
minian_path = "."
sys.path.insert(0, minian_path)
from minian.utilities import (
    match_information,
    match_path
)

In [ ]:
dpath="/N/project/Cortical_Calcium_Image/Miniscope data/12.2022_Sixth_group/AA034_D7/2022_12_10/10_41_39/Miniscope_2"
inputfilename = "2022-12-10_10h41m_Subject YC_AA034.txt"
mouseID, day, session = match_information(dpath)
mouse_path, video_path = match_path(dpath)
outputfilename = "behavior_ms.csv"
timestamp_path=os.path.join(video_path, "timeStamps.csv")

In [ ]:
timestamp_pd=pd.read_csv(timestamp_path)

In [ ]:
def reformat(data_array,col_name, time_pd):
#     output = pd.Dataframe()
    j = 0
    flag = 0
    output = pd.concat([time_pd],axis=1)
    res = []
    for i in range(0,len(output['Time Stamp (ms)'])):
#         print(type(output['Time Stamp (ms)'][i]))
#         print(data_array[j]*1000)
        if float(output['Time Stamp (ms)'][i]) <= data_array[j]*1000 and float(output['Time Stamp (ms)'][i+1])>data_array[j]*1000:
#             print(str(i)+" big")
            if data_array[j]==0 and flag==0:
                flag=1
                j=j+1
                res.append(1)
                continue
            elif data_array[j]==0:
                break    
            res.append(1)
            j=j+1
        else:
#             print(str(i)+ " small")
            res.append(0)
#     print(output['Time Stamp (ms)'])
    while len(res)<len(output['Time Stamp (ms)']):
        res.append(0)
#     print(j)
    
    output[col_name]=res
#     print(res)
#     print(output)
    return output

In [ ]:
alp_number: int
ialp_number: int
rnfs_number: int
X_array = []
Y_array = []
Z_array = []
file_path = os.path.join(mouse_path,inputfilename)
behavior_timeStamp = list(csv.reader(open(file_path)))
# for line in behavior_timeStamp:
#     print(line)
alp_number = int(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[13][0])[0]))
ialp_number = int(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[21][0])[0]))
rnfs_number = int(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[14][0])[0]))
# alp=re.findall(r"\d+\.?\d*", )
print(alp_number)
print(ialp_number)
print(rnfs_number)
print(behavior_timeStamp[36][0])  # X: reinforcement
for i in range(37, 237):
    for j in range(1, 6):
        X_array.append(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[i][0])[j]))
# print(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[237][0])[1]))
X_array.append(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[237][0])[1]))
print(behavior_timeStamp[238][0])  # Y: active lever press
for i in range(239, 439):
    for j in range(1, 6):
        Y_array.append(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[i][0])[j]))
Y_array.append(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[439][0])[1]))
print(behavior_timeStamp[440][0])  # Z: inactive lever press
for i in range(441, 641):
    for j in range(1, 6):
        Z_array.append(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[i][0])[j]))
Z_array.append(float(re.findall(r"\d+\.?\d*", behavior_timeStamp[641][0])[1]))


res = reformat(X_array,"reinforcement" ,timestamp_pd)
res = reformat(Y_array, "ALP",res)
res = reformat(Z_array,"IALP",res)

res.to_csv(os.path.join(mouse_path,mouseID+"_"+day+"_"+"behavior_ms.csv"), index = None)
